# C++ Builder & Runner

In [ ]:
exec('import os,sys,subprocess,importlib.util,pathlib\nif not importlib.util.find_spec("requests"):subprocess.check_call([sys.executable,"-m","pip","install","requests"])\nimport requests\ndef g(u,r,p):\n os.makedirs(p,exist_ok=True)\n for i in requests.get(f"https://api.github.com/repos/{u}/{r}/contents/{p}").json():\n  if i["type"]=="dir":g(u,r,i["path"])\n  else:open(os.path.join(p,i["name"]),"wb").write(requests.get(i["download_url"]).content)\nif not pathlib.Path(".builder").exists():print("⬇️ Downloading .builder...");g("IGORSVOLOHOVS","TaskManager",".builder")')

%run .builder/build.py
%run .builder/agent_rules/rules.py
%run .builder/tools/format.py
%run .builder/tools/documentation.py
%run .builder/project_templates/TDD.py

In [ ]:
# project_tdd_setup('.')

In [ ]:
configs = [
    # Обычный Debug
    # {
    #     'name': 'GCC',   
    #     'cc': 'g++',     
    #     'type': 'debug', 
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -g -O0'
    # },
    # # Обычный Release
    # {
    #     'name': 'CLANG', 
    #     'cc': 'clang++', 
    #     'type': 'release',
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -O3 -Wall'
    # },
    # Coverage Configuration
    {
        'name': 'GCC-Cov',
        'cc': 'g++',
        'type': 'test',
        # Добавил -std=c++23
        'flags': '-std=c++23 -g -O0',
        'cov_flags': '--coverage',
        'cov_tool': 'lcov'
    }
]

for c in configs: run_pipeline(c)

In [ ]:
# gen_docs()

In [ ]:
# format_code()

In [ ]:
# clean()